In [1]:
# from google.cloud import bigquery

# GCP_PROJECT = "cedar-heaven-387214"
# BQ_DATASET = "PLACEHOLDER"

# client = bigquery.Client()

# # Create the BigQuery dataset
# dataset_id = f"{GCP_PROJECT}.{BQ_DATASET}"
# dataset = bigquery.Dataset(dataset_id)
# dataset.location = "US"
# try:
#     dataset = client.create_dataset(dataset, timeout=30)  # Make an API request.
#     print("Created dataset {}.{}".format(client.project, dataset.dataset_id))
# except exceptions.Conflict:
#     print("Dataset already exists - choose a new name if the dataset is not under your control")
#     pass

In [2]:
# from google.cloud import storage

# GCP_REGION = "northamerica-northeast2"
# GCP_BUCKET = "amazon_shoes_review"

# storage_client = storage.Client()

# try:
#     bucket = storage_client.bucket(GCP_BUCKET)
#     bucket.storage_class = "STANDARD"
#     new_bucket = storage_client.create_bucket(bucket, location=GCP_REGION)
# except exceptions.Conflict:
#     print("Bucket already exists - choose a new name if the bucket is not under your control")
#     pass

In [3]:
# from google.cloud import aiplatform

# GCP_PROJECT = "cedar-heaven-387214"
# GCP_REGION = "northamerica-northeast2"
# EXPERIMENT_NAME = "example"

# # Vertex AI initialization
# aiplatform.init(
#     project=GCP_PROJECT,
#     location=GCP_REGION,
#     experiment=EXPERIMENT_NAME,
# )

Uncomment and run the cell below if needed for google cloud

In [24]:
# %cd Amazon-Reviews-Capstone

[Errno 2] No such file or directory: 'Amazon-Reviews-Capstone'
/home/jupyter/Amazon-Reviews-Capstone


In [25]:
%%time
# Loading the data
import pandas as pd

# Grabbing the bucket where the shoes are if this is first load
# bucket_name = 'amazon_shoes_review'
# file_path = 'gs://amazon_shoes_review/Shoes.tsv'

file_path = 'Shoes.tsv'
print(file_path)
df = pd.read_csv(file_path, sep='\t', on_bad_lines='skip')

Shoes.tsv
CPU times: user 28.5 s, sys: 3.03 s, total: 31.5 s
Wall time: 31.5 s


In [ ]:
# Load the pickled object
with open('shoes.pkl', 'rb') as f:
    df = pickle.load(f)

# Print the loaded object
print(df)

In [ ]:
import pickle

with open('shoes.pkl', 'wb') as f:
    pickle.dump(df, f)

In [ ]:
# %%time
# from google.cloud import storage

# # Create a client instance
# storage_client = storage.Client()

# # Specify your bucket and CSV file name
# bucket = 'amazon_shoes_review'
# csv = 'Shoes.tsv'

# # Get the bucket and file objects
# bucket = storage_client.get_bucket(bucket)
# blob = bucket.blob(csv)

# # Download the CSV file to a local temporary file
# local_file_path = 'Shoes.tsv'
# blob.download_to_filename(local_file_path)

# # Read the CSV file using pandas or any other library of your choice
# import pandas as pd

# df = pd.read_csv(file_path, sep='\t', on_bad_lines='skip')

In [ ]:
print(df.head())

In [ ]:
df.head()

In [ ]:
import os
import pandas as pd
# pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer

In [ ]:
embedder = SentenceTransformer('bert-base-cased')

In [ ]:
product_names = df['product_title']
product_names

In [ ]:
corpus = list(product_names)
corpus = corpus[0:50000]
corpus

In [ ]:
%%time
corpus_embeddings = embedder.encode(corpus)

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
%%time
num_clusters = 2
clustering_model = KMeans(n_clusters=num_clusters)
clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_

In [ ]:
cluster_df = pd.DataFrame(corpus, columns = ['corpus'])
cluster_df['cluster'] = cluster_assignment
cluster_df

In [ ]:
common_shoes = ['running shoe', 'walking shoe', 'flip', 'sandal', 'sneaker'] #flip for flip flop

In [ ]:
label_shoes = df['product_title'].str.contains('|'.join(common_shoes), case=False, na=False)
df_shoes=df[label_shoes]

In [ ]:
test = ['flip flop']
label_test = df['product_title'].str.contains('|'.join(test), case=False, na=False)
df_test=df[label_test]

In [ ]:
df_test

In [ ]:
common_others = [' bag ', 'sunglasses', 'wallet', 'ray ban', 'rayban', 'tote', 'handbag', 'purse', 'clutch'] 

In [ ]:
label_others = df['product_title'].str.contains('|'.join(common_others), case=False, na=False)
df[label_others]

In [ ]:
df_shoes[label_others]

In [ ]:
# creating a dataframe with just product names
df_names = df['product_title']
df_others = pd.DataFrame(df_names[label_others]) #pd.Dataframe is needed to add the column label after
df_shoes = pd.DataFrame(df_names[label_shoes])
df_others['label'] = 'others'
df_shoes['label'] = 'shoes'

In [ ]:
# combining and removing products that have shoes and others tag
combined_df = pd.concat([df_others, df_shoes])
filtered_df = combined_df[~(label_others & label_shoes)]

In [ ]:
common_shoes = ['running shoe', 'walking shoe', 'flip', 'sandal', 'sneaker', 'boot', 'crocs', 'skateboarding shoe', 'clog', 'hiking shoe', 'slipper', 'flat', 'pump', 'shoe'] #flip for flip flop

In [ ]:
label_shoes = df['product_title'].str.contains('|'.join(common_shoes), case=False, na=False)
df_shoes=df[label_shoes]

In [ ]:
# Create a boolean mask indicating rows in df1 that are not present in df2
mask = ~df.isin(df_shoes)

# Filter df1 using the mask to get rows not in df2
result = df[mask]

In [ ]:
# Print the result

list(result.dropna()['product_title'].unique())

In [ ]:
df_shoes['product_title']

In [ ]:
clustered_sentences = [[] for i in range(num_clusters)]
for sentence_id, cluster_id in enumerate(cluster_assignment):
    clustered_sentences[cluster_id].append(corpus[sentence_id])
    
for i, cluster in enumerate(clustered_sentences):
    print("Cluster ", i+1)
    print(cluster)
    print("")

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from wordcloud import WordCloud

In [ ]:
def word_cloud(pred_df, label):
    wc = ' '.join([text for text in pred_df['corpus'][pred_df['cluster'] == label]])
    wordcloud = WordCloud(width = 800, height = 500, random_state = 20, max_font_size = 110).generate(wc)
    fig7 = plt.figure(figsize=(10,7))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')

In [ ]:
word_cloud(cluster_df, 0)

In [ ]:
word_cloud(cluster_df, 1)

In [ ]:
labeled_embeddings = embedder.encode(filtered_df)

In [ ]:
%%time
corpus_embeddings = embedder.encode(corpus)

In [ ]:
%%time
num_clusters = 2
clustering_model = KMeans(n_clusters=num_clusters)
clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_